<a href="https://twitter.com/kESXAFKuPdhrfHb" target="_parent"><img src="https://github.com/IMIYOU/gardendream/blob/master/QR.png?raw=true" alt="Open In Colab"/></a>

# 【盜臉駭客製作-手機換臉繁中版】

此為本群獨立原創Colab,僅限科學用途,一切非法行為本群皆不負責！
(如有任何問題,請掃右上QR碼/TG私訊群主,將有專人為您服務)


# 本軟體重點
*   臉部提取功能正常！
*   與PC端不同,訓練不可直接預覽！
*   您可以使用Google驅動導入/導出工作區！
*   使用“管理工作區”塊中可以執行的工作區導入/導出和其他操作！
*   本軟體單次運作最多12小时，盜臉駭客手機版Colab在訓練模式下將備份您的工作區！
*   本軟體可刷出多款高效率AI顯卡,P100為頂級16G,優於市面上多數卡
*   此為連接雲端電腦操作,非自用電腦,操作性上有多種侷限,僅推薦輕量玩家使用,若想有最好體驗,請購買【盜臉駭客-繁中電腦版】


# 【第一步】查看分配到的GPU(顯卡)

*   Google Colab可以為您提供一個特斯拉圖形顯卡: K80(11G), T4(16G), P4(8G) or P100(16G)
*   在使用盜臉駭客換臉之前，您可以在這裡檢查配備的GPU型號



In [0]:
!nvidia-smi

#【第二步】連接驅動Google雲端

In [0]:
#@title 開始掛載 
#連接Google雲盘
#點擊連接授權，複製授權碼，粘貼到文本框內，按后按ENTER鍵。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#創建程式目錄，並且進入目錄
%cd /content/


# 【第三步】安裝盜臉駭客及運行環境

* 所有安裝都是自動的,請勿有多餘動作

In [0]:
#@title 安裝HackFake雲端程式

Mode = "開始安裝" #@param ["開始安裝"]

#創建HackFake目錄，並且進入目錄
%cd /content/

from pathlib import Path
if (Mode == "開始安裝"):
  !git clone https://github.com/takeyouface/HackFake001.git



print("Done!")

In [0]:
#@title 安裝運行環境
!pip uninstall -y tensorflow
!pip install tqdm
!pip install numpy==1.17.0
!pip install h5py==2.9.0
!pip install opencv-python==4.1.0.25
!pip install ffmpeg-python==0.1.17
!pip install scikit-image==0.14.2
!pip install scipy==1.4.1
!pip install colorama
!pip install tensorflow-gpu==1.14.0
!pip install --upgrade scikit-image
!apt-get install cuda-10-0

#【第四步】管理工作區(workspace)



*   您可以導入/導出工作區或單個數據，如使用Google Drive的模型
*   您可以清除所有工作區或刪除其中的一部分



In [0]:
#@title 從云盤根目錄減壓到WORKSPACE工作區

Mode = "models" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "model.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content/drive/My\ Drive", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/drive/My\ Drive/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/drive/My\ Drive/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/drive/My\ Drive/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/drive/My\ Drive/workspace/data_dst", "aligned")
elif Mode == "models":
  zip_and_copy("/content/drive/My\ Drive/workspace", "model")
  
print("Done!")



In [0]:
#@title 壓縮WORKSPACE工作區目錄到云盤根目錄 { form-width: "30%" }
Mode = "data_src" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "merged", "models", "result video"]
Archive_name = "data_src.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  zip_cmd="-r -q "+Archive_name+" "
  
  %cd $path
  zip_cmd+=mode
  !zip $zip_cmd
  copy_cmd = " "+Archive_name+"  /content/drive/My\ Drive/"
  !cp $copy_cmd
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content/drive/My\ Drive", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/drive/My\ Drive/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/drive/My\ Drive/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/drive/My\ Drive/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/drive/My\ Drive/workspace/data_dst", "aligned")
elif Mode == "merged":
  zip_and_copy("/content/drive/My\ Drive/workspace/data_dst", "merged")
elif Mode == "models":
  zip_and_copy("/content/drive/My\ Drive/workspace", "model")
elif Mode == "result video":
  !cp /content/drive/My\ Drive/workspace/result.mp4 /content/drive/My\ Drive/
  
print("Done!")


In [0]:
#@title 刪除并重建WORKSPACE工作區目錄
Mode = "Delete and recreate workspace" #@param ["Delete and recreate workspace", "Delete models", "Delete data_src", "Delete data_src aligned", "Delete data_src video", "Delete data_dst", "Delete data_dst aligned", "Delete merged frames"]

%cd "/content/drive/My Drive" 

if Mode == "Delete and recreate workspace":
  cmd = "rm -r /content/drive/My\ Drive/workspace ; mkdir /content/drive/My\ Drive/workspace; mkdir /content/drive/My\ Drive/workspace/data_src; mkdir /content/drive/My\ Drive/workspace/data_src/aligned; mkdir /content/drive/My\ Drive/workspace/data_dst; mkdir /content/drive/My\ Drive/workspace/data_dst/aligned; mkdir /content/drive/My\ Drive/workspace/model"  
elif Mode == "Delete models":
  cmd = "rm -r /content/drive/My\ Drive/workspace/model/*"
elif Mode == "Delete data_src":
  cmd = "rm /content/drive/My\ Drive/workspace/data_src/*.png || rm -r /content/drive/My\ Drive/workspace/data_src/*.jpg"
elif Mode == "Delete data_src aligned":
  cmd = "rm -r /content/drive/My\ Drive/workspace/data_src/aligned/*"
elif Mode == "Delete data_src video":
  cmd = "rm -r /content/drive/My\ Drive/workspace/data_src.*"
elif Mode == "Delete data_dst":
  cmd = "rm /content/drive/My\ Drive/workspace/data_dst/*.png || rm /content/drive/My\ Drive/workspace/data_dst/*.jpg"
elif Mode == "Delete data_dst aligned":
  cmd = "rm -r /content/drive/My\ Drive/workspace/data_dst/aligned/*"
elif Mode == "Delete merged frames":
  cmd = "rm -r /content/drive/My\ Drive/workspace/data_dst/merged"
  
!$cmd
print("Done!")

# 【第五步】人臉提取、排序和人臉素材工具
* 為SRC或DST影片轉換圖片
* 對SRC或DST影片進行降噪
* 使用其中一個檢測器提取人臉，（S3FD-推薦），如果需要，可以使用修復提取圖片
* 提取並手動排序後将工作區導出到Google Drive（在“管理工作區”中）
* 你可以使用FacesetEnhancer增强你的faceset增强圖片將取代您的“aligned”資料
* 選擇之素材,請不要使用過長的影片(手機版推薦五分鐘以下)不然會提取到死
* (購買盜臉駭客SRC.DST素材者,請直接跳過此階段,已經幫您處理好可直接使用)


In [0]:
#@title 視頻轉換成圖片
Video = "data_src" #@param ["data_src", "data_dst"]

%cd "/content"

cmd = "HackFake001/main.py videoed extract-video"

if Video == "data_dst":
  cmd+= " --input-file drive/My\ Drive/workspace/data_dst.* --output-dir drive/My\ Drive/workspace/data_dst/"
else:
  cmd+= " --input-file drive/My\ Drive/workspace/data_src.* --output-dir drive/My\ Drive/workspace/data_src/"
  
!python $cmd

In [0]:
#@title 圖片降噪
Data = "data_src" #@param ["data_src", "data_dst"]
Factor = 1 #@param {type:"slider", min:1, max:20, step:1}

cmd = "HackFake001/main.py videoed denoise-image-sequence --input-dir drive/My\ Drive/workspace/"+Data+" --factor "+str(Factor)

%cd "/content"
!python $cmd

In [0]:
#@title 提取人臉素材
Data = "data_src" #@param ["data_src", "data_dst"]
Detector = "S3FD" #@param ["S3FD", "S3FD (整張臉)"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
dbg = " --output-debug" if Debug else " --no-output-debug"

folder = "drive/My\ Drive/workspace/"+Data
folder_aligned = folder+"/aligned"

cmd = "HackFake001/main.py extract --input-dir "+folder+" --output-dir "+folder_aligned
cmd+=" --detector "+detect_type+" --force-gpu-idxs 0"+dbg

if "整張臉" in Detector:
  cmd+=" --face-type whole_face" 
%cd "/content"
!python $cmd

In [0]:
#@title 人臉素材排序篩查
Data = "data_src" #@param ["data_src", "data_dst"]
sort_type = "hist" #@param ["blur", "face-yaw", "face-pitch", "face-source-rect-size", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final", "final-faster", "absdiff"]

cmd = "HackFake001/main.py sort --input-dir drive/My\ Drive/workspace/"+Data+"/aligned --by "+sort_type

%cd "/content"
!python $cmd

In [0]:
#@title 人臉素材細節提升
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/drive/My\ Drive/workspace/"+Data+"/aligned"
cmd = "/content/HackFake001/main.py facesettool enhance --input-dir "+data_path
!python $cmd

In [0]:
#@title 人臉素材打包
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/drive/My\ Drive/workspace/"+Data+"/aligned"
cmd = "/content/HackFake001/main.py util --pack-faceset --input-dir "+data_path
!python $cmd

In [0]:
#@title 人臉素材解包
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/drive/My\ Drive/workspace/"+Data+"/aligned"
cmd = "/content/HackFake001/main.py util --unpack-faceset --input-dir "+data_path
!python $cmd

In [0]:
#@title 為素材應用或者移除訓練好的XSeg遮罩（記得解包人臉素材后操作）
Mode = "Remove mask" #@param ["Apply mask", "Remove mask"]
Data = "data_src" #@param ["data_src", "data_dst"]

main_path = "/content/HackFake001/main.py "
data_path = "/content/drive/My\ Drive/workspace/"+Data+"/aligned "
mode_arg = "apply " if Mode == "Apply mask" else "remove "
cmd = main_path+"xseg "+mode_arg+"--input-dir "+data_path
cmd += "--model-dir /content/drive/My\ Drive/workspace/model" if mode_arg == "apply " else ""

!python $cmd

# 【第六步】訓練階段

* 訓練模式分為換臉模型以及XSeg遮罩模型，根據自己需求選擇！
* 在Model字段上選擇模型
* 如果轉到文件管理器中的“模型”文件夾並雙擊preview.jpg文件，则可以手動查看預覽
* 此外，還可以在“管理工作區”中手動導出工作區，訓練XSeg模型时要與你的人脸模型訓練範圍相同
* Xseg僅為穩定臉部效果使用,非臉部模型,手機版無法使用繪畫功能,僅在有購買群主SRC.DST素材,可開啟此選項訓練,出片效果加倍!

In [0]:
#@title 模型訓練
#連接驅動Google雲端
#點擊鏈接授權，複製授權碼，填入文本框，然後按ENTER鍵開始。

Model = "advanced" #@param ["advanced", "XSeg"]
Silent_Start = True #@param {type:"boolean"}

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 安裝HackFake程式
%cd /content/
!git clone https://github.com/takeyouface/HackFake001.git

# 安裝運行環境
!pip uninstall -y tensorflow
!pip install tqdm
!pip install numpy==1.17.0
!pip install h5py==2.9.0
!pip install opencv-python==4.1.0.25
!pip install ffmpeg-python==0.1.17
!pip install scikit-image==0.14.2
!pip install scipy==1.4.1
!pip install colorama
!pip install tensorflow-gpu==1.14.0
!pip install --upgrade scikit-image
!apt-get install cuda-10-0

# 開始訓練模型
cmd ="HackFake001/main.py train --training-data-src-dir drive/My\ Drive/workspace/data_src/aligned --training-data-dst-dir drive/My\ Drive/workspace/data_dst/aligned --model-dir drive/My\ Drive/workspace/model --model " + Model +" --no-preview"

if Model == "Quick96":
  cmd+= " --pretrained-model-dir pretrain_Q96"

if Silent_Start:
  cmd+= " --silent-start"

else:
  train_cmd = (cmd)

!python $cmd

# 【最終階段】影片合成

In [0]:
#@title 合成圖片
Model = "advanced" #@param ["advanced" ]

cmd = "HackFake001/main.py merge --input-dir drive/My\ Drive/workspace/data_dst --output-dir drive/My\ Drive/workspace/data_dst/merged --output-mask-dir drive/My\ Drive/workspace/data_dst/merged_mask --aligned-dir drive/My\ Drive/workspace/data_dst/aligned --model-dir drive/My\ Drive/workspace/model --model "+Model

%cd "/content"
!python $cmd

In [0]:
#@title 合成影片并複製到云盤根目錄
Mode = "result video" #@param ["result video", "result_mask video"]
Copy_to_Drive = True #@param {type:"boolean"}


if Mode == "result video":
  !python HackFake001/main.py videoed video-from-sequence --input-dir drive/My\ Drive/workspace/data_dst/merged --output-file drive/My\ Drive/workspace/result.mp4 --reference-file drive/My\ Drive/workspace/data_dst.mp4 --include-audio
  if Copy_to_Drive:
    !cp /content/drive/My\ Drive/workspace/result.mp4 /content/drive/My\ Drive/
elif Mode == "result_mask video":
  !python HackFake001/main.py videoed video-from-sequence --input-dir drive/My\ Drive/workspace/data_dst/merged_mask --output-file drive/My\ Drive/workspace/result_mask.mp4 --reference-file drive/My\ Drive/workspace/data_dst.mp4
  if Copy_to_Drive:
    !cp /content/drive/My\ Drive/workspace/result_mask.mp4 /content/drive/My\ Drive/
